# Toboggan Trajectory

## Part 1: How many trees do we crash into? 😅

Looks like we're dealing with a 2D array today. Let's get it loaded up.

In [1]:
#!fsharp
#!value --name input --from-file input.txt

In [1]:
#!fsharp
#!share input --from value

let trees = 
  input.Split("\n", System.StringSplitOptions.RemoveEmptyEntries)
  |> array2D
  |> Array2D.map ((=) '#')

The only tricky thing is that the array repeats horizontally. I feel like the more comprehensive solution would be to make some sort of `ToroidalMatrix<'a>` type, but we're going to have to write a `traverse` function anyway so might as well just stick the modulo operator `%` in there while we're at it.

In [1]:
#!fsharp
let traverse arr (dx, dy) =
  if (dx < 0 || dy < 0) then invalidArg "vector" "Vector must be positive"
  let (lenX, lenY) = (Array2D.length2 arr, Array2D.length1 arr)
  (0,0) |> Seq.unfold (
    function
    | (x, y) when y < lenY -> Some (arr.[y, x % lenX], (x + dx, y + dy))
    | _ -> None
  )

traverse trees (3, 1) 
|> Seq.filter id
|> Seq.length 

228

## Part 2

Hey! Our bet paid off in writing `traverse` to take a vector argument. Second solution's just a `map`-`reduce` away.

In [1]:
#!fsharp
let vectors = [
  (1,1);
  (3,1);
  (5,1);
  (7,1);
  (1,2)
] 
let treeCounts = vectors |> List.map (traverse trees >> Seq.filter id >> Seq.length)
let product = treeCounts |> List.map int64 |> List.reduce (*)
$"%A{treeCounts} = {product}"

[84; 228; 89; 100; 40] = 6818112000

# Bonus Stage

Here's that `ToroidalMatrix` idea I mentioned. Think it ends up not really being worth it for this, but it's an interesting concept.

In [1]:
#!fsharp
module Seq =
  let repeatInfinite sequence = seq { while true do yield! sequence }

type Toroidal<'a> (seq2d: 'a seq seq) =
  member this.Item 
    with get(x,y) = seq2d |> Seq.item x |> Seq.repeatInfinite |> Seq.item y
  member this.Height = seq2d |> Seq.length

(Toroidal [[1;2;3]; [4;5;6]]).[1,4]

5